<a href="https://colab.research.google.com/github/tomfox1/Predictive-Modelling-Challenge-/blob/master/Data_Wrangling_Ideas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add your ideas for data wrangling in a cell below. Add code if you would like or just ideas for features/ways to clean the dataset.




1.   Population around each well, higher pop-density would mean higher wear
2.   Does elevation interact with condition of well
3.   Airpollution? compare polution in the area to the longevity of the well 
4.   Does a certain installer have longer lasting wells? 





let's get a majority <-?

In [0]:
###!!! DONT JUST COPY AND PASTE

def MrClean(df):
    df_t= df
    df_t['gps_height'].replace(0.0, np.nan, inplace=True)
    df_t['population'].replace(0.0, np.nan, inplace=True)
    df_t['amount_tsh'].replace(0.0, np.nan, inplace=True)
    df_t['gps_height'].fillna(df_t.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
    df_t['gps_height'].fillna(df_t.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
    df_t['gps_height'].fillna(df_t['gps_height'].mean(), inplace=True)
    df_t['population'].fillna(df_t.groupby(['region', 'district_code'])['population'].transform('median'), inplace=True)
    df_t['population'].fillna(df_t.groupby(['region'])['population'].transform('median'), inplace=True)
    df_t['population'].fillna(df_t['population'].median(), inplace=True)
    df_t['amount_tsh'].fillna(df_t.groupby(['region', 'district_code'])['amount_tsh'].transform('median'), inplace=True)
    df_t['amount_tsh'].fillna(df_t.groupby(['region'])['amount_tsh'].transform('median'), inplace=True)
    df_t['amount_tsh'].fillna(df_t['amount_tsh'].median(), inplace=True)
    features=['amount_tsh', 'gps_height', 'population']
    scaler = MinMaxScaler(feature_range=(0,20))
    df_t[features] = scaler.fit_transform(df_t[features])
    df_t['longitude'].replace(0.0, np.nan, inplace=True)
    df_t['latitude'].replace(0.0, np.nan, inplace=True)
    df_t['construction_year'].replace(0.0, np.nan, inplace=True)
    df_t['latitude'].fillna(df_t.groupby(['region', 'district_code'])['latitude'].transform('mean'), inplace=True)
    df_t['longitude'].fillna(df_t.groupby(['region', 'district_code'])['longitude'].transform('mean'), inplace=True)
    df_t['longitude'].fillna(df_t.groupby(['region'])['longitude'].transform('mean'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['region', 'district_code'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['region'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t.groupby(['district_code'])['construction_year'].transform('median'), inplace=True)
    df_t['construction_year'].fillna(df_t['construction_year'].median(), inplace=True)
    df_t['date_recorded'] = pd.to_datetime(df_t['date_recorded'])
    df_t['years_service'] = df_t.date_recorded.dt.year - df_t.construction_year
   
    garbage=['wpt_name','num_private','subvillage','region_code','recorded_by','management_group',
         'extraction_type_group','extraction_type_class','scheme_name','payment',
        'quality_group','quantity_group','source_type','source_class','waterpoint_type_group',
        'ward','public_meeting','permit','date_recorded','construction_year']
    df_t.drop(garbage,axis=1, inplace=True)
    df_t.waterpoint_type = df_t.waterpoint_type.str.lower()
    df_t.funder = df_t.funder.str.lower()
    df_t.basin = df_t.basin.str.lower()
    df_t.region = df_t.region.str.lower()
    df_t.source = df_t.source.str.lower()
    df_t.lga = df_t.lga.str.lower()
    df_t.management = df_t.management.str.lower()
    df_t.quantity = df_t.quantity.str.lower()
    df_t.water_quality = df_t.water_quality.str.lower()
    df_t.payment_type=df_t.payment_type.str.lower()
    df_t.extraction_type=df_t.extraction_type.str.lower()
    df_t["funder"].fillna("other", inplace=True)
    df_t["scheme_management"].fillna("other", inplace=True)
    df_t["installer"].fillna("other", inplace=True)
    
    #further spacial/location information
    #https://www.kaggle.com/c/sf-crime/discussion/18853
    
    return df_t

In [0]:
def con_matrix_analysis(model, feature_matrix, target_vector):
  from sklearn.metrics import classification_report, confusion_matrix
  
  x = model.predict(feature_matrix)
  y = target_vector
  
  print(classification_report(y, x,
        target_names=['Functional', 'Needs Repair', 'Not-Functional']))

  con_matrix = pd.DataFrame(confusion_matrix(y, x), 
             columns=['Predicted Functional', 'Predicted Needs Repair', 'Predicted Not-Functional'], 
             index=['Actual Functional', 'Actual Needs Repair', 'Actual Not-Functional'])
                            
  sns.heatmap(data=con_matrix, cmap='cool')
  plt.show();
  return con_matrix

In [0]:
def reverse_cardinality_check(n, df):
  """
  Given a cardinality limit (n) and a dataframe this function will search the
  dataframe for features above the cardinality limit, then create a dict
  from the results
  """
  
  feature_list = []
  
  cardinality_value = []
  
  for _ in range(len(df.columns)):
    if len(df[df.columns[_]].value_counts()) > n:
      
      feature_list.append(df.columns[_])
      
      cardinality_value.append(len(df[df.columns[_]].value_counts()))
                               
        
  feature_dict = dict(zip(feature_list, cardinality_value))
  
  return feature_dict

In [0]:
def wrangle(X):
    # Create copy of dataframe to avoid copy warning
    X = X.copy()
    
    # Some features have missing data showing as 0 that need to be changed to nan
    X['gps_height'] = X['gps_height'].replace(0.0, np.nan)
    X['longitude'] = X['longitude'].replace(0.0, np.nan)
    X['latitude'] = X['latitude'].replace(0.0, np.nan)
    X['construction_year'] = X['construction_year'].replace(0.0, np.nan)
    X['population'] = X['population'].replace(0.0, np.nan)
    X['amount_tsh'] = X['amount_tsh'].replace(0.0, np.nan)
    
    # gps_height: replace nan values with the mean of the smallest geographical region possible
    # Excluding subvillage due to missing values
    X['gps_height'].fillna(X.groupby(['ward'])['gps_height'].transform('mean'), inplace=True)
    X['gps_height'].fillna(X.groupby(['district_code'])['gps_height'].transform('mean'), inplace=True)
    X['gps_height'].fillna(X.groupby(['region_code'])['gps_height'].transform('mean'), inplace=True)
    X['gps_height'].fillna(X['gps_height'].mean(), inplace=True)
    
    # longitude: replace nan values with the mean of the smallest geographical region possible
    # Excluding subvillage due to missing values
    X['longitude'].fillna(X.groupby(['ward'])['longitude'].transform('mean'), inplace=True)
    X['longitude'].fillna(X.groupby(['district_code'])['longitude'].transform('mean'), inplace=True)
    X['longitude'].fillna(X.groupby(['region_code'])['longitude'].transform('mean'), inplace=True)
    X['longitude'].fillna(X['longitude'].mean(), inplace=True)
    
    # latitude: replace nan values with the mean of the smallest geographical region possible
    # Excluding subvillage due to missing values
    X['latitude'].fillna(X.groupby(['ward'])['latitude'].transform('mean'), inplace=True)
    X['latitude'].fillna(X.groupby(['district_code'])['latitude'].transform('mean'), inplace=True)
    X['latitude'].fillna(X.groupby(['region_code'])['latitude'].transform('mean'), inplace=True)
    X['latitude'].fillna(X['latitude'].mean(), inplace=True)
    
    # population: replace nan values with the mean of the smallest geographical region possible
    # Excluding subvillage due to missing values
    X['population'].fillna(X.groupby(['ward'])['population'].transform('median'), inplace=True)
    X['population'].fillna(X.groupby(['district_code'])['population'].transform('median'), inplace=True)
    X['population'].fillna(X.groupby(['region_code'])['population'].transform('median'), inplace=True)
    X['population'].fillna(X['population'].median(), inplace=True)
    
    # population: replace nan values with the mean of the smallest geographical region possible
    # Excluding subvillage due to missing values
    X['amount_tsh'].fillna(X.groupby(['ward'])['amount_tsh'].transform('median'), inplace=True)
    X['amount_tsh'].fillna(X.groupby(['district_code'])['amount_tsh'].transform('median'), inplace=True)
    X['amount_tsh'].fillna(X.groupby(['region_code'])['amount_tsh'].transform('median'), inplace=True)
    X['amount_tsh'].fillna(X['amount_tsh'].median(), inplace=True)
    
    # construction_year: replace nan values with the mean of the smallest geographical region possible
    # Excluding subvillage due to missing values
    X['construction_year'].fillna(X.groupby(['ward'])['construction_year'].transform('median'), inplace=True)
    X['construction_year'].fillna(X.groupby(['district_code'])['construction_year'].transform('median'), inplace=True)
    X['construction_year'].fillna(X.groupby(['region_code'])['construction_year'].transform('median'), inplace=True)
    X['construction_year'].fillna(X['construction_year'].median(), inplace=True)
    
    # Convert date to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract datetime data
    X['year_recorded'] = X['date_recorded'].dt.year
    
    # Drop duplicate or unnecessary features
    X = X.drop(columns=['recorded_by', 'quantity_group', 'date_recorded', 'wpt_name', 'num_private', 'subvillage',
                       'region_code', 'management_group', 'extraction_type_group', 'extraction_type_class',
                       'scheme_name', 'payment', 'water_quality', 'source_type', 'source_class', 'waterpoint_type_group',
                       'ward', 'public_meeting', 'permit'])
    
    # Several categorical features have values showing as '0'
    # Replace '0' with nan
    categoricals = X.select_dtypes(exclude='number').columns.tolist()
    X[categoricals] = X[categoricals].replace('0', np.nan)
    
    # Convert to lowercase to collapse duplicates
    X['waterpoint_type'] = X['waterpoint_type'].str.lower()
    X['funder'] = X['funder'].str.lower()
    X['basin'] = X['basin'].str.lower()
    X['region'] = X['region'].str.lower()
    X['source'] = X['source'].str.lower()
    X['lga'] = X['lga'].str.lower()
    X['management'] = X['management'].str.lower()
    X['quantity'] = X['quantity'].str.lower()
    X['quality_group'] = X['quality_group'].str.lower()
    X['payment_type'] = X['payment_type'].str.lower()
    X['extraction_type'] = X['extraction_type'].str.lower()
    
    # Replace nan values with 'other'
    X["funder"].fillna("other", inplace=True)
    X["scheme_management"].fillna("other", inplace=True)
    X["installer"].fillna("other", inplace=True)
    
    # Replace any remaining nan values with 'other'
    X = X.replace(np.nan, 'other')
    
    return X

In [0]:
# These functions do an excellent job cleaning up the data and also do some conservative feature engineering

def cleanup1(X):
  X2 = X.copy()

  
  X2 = X2.fillna('unknown')
  dates = pd.to_datetime(X2.date_recorded)
  year2000 = pd.to_datetime('2000-01-01')
  years = [i.days/365 for i in (dates - year2000)]
  X2.date_recorded = years

  X2.region_code = X2.region_code.astype('str')
  X2.district_code = X2.district_code.astype('str')

  X2 = X2.drop(columns='recorded_by')

  type_dict = {'amount_tsh':'float64','date_recorded':'float64',
                 'gps_height':'float64',
                 'longitude':'float64',
                 'latitude':'float64',
                 'num_private':'float64',
                 'population':'float64',
                 'construction_year':'float64',
                 'public_meeting':'str',
                 'permit':'str'}
    
  X2 = X2.astype(dtype = type_dict)
    
  return X2

from sklearn.impute import MissingIndicator

def cleanup2(X):
  X2 = X.copy()
    
    # I make a list of the numerical columns and a dict of their 
    # garbage values that really should be nulls
  numericals = ['amount_tsh',
                    'date_recorded',
                    'gps_height',
                    'longitude',
                    'latitude',
                    'num_private',
                    'population',
                    'construction_year']

  null_values = {'amount_tsh':0,
                     'date_recorded':0,
                     'gps_height':0,
                     'longitude':0,
                     'latitude':-2.000000e-08,
                     'num_private':0,
                     'population':0,
                     'construction_year':0}

    # I replace all garbage values with NANs.
  for feature, null in null_values.items():
        X2[feature] = X2[feature].replace(null, np.nan)

    # construction_year occasionally claims years far in the future, and 
    # could presumably also contain years way in the past.  I'll turn anything
    # not between 1960 and 2019 into a NAN.
  X2['construction_year'] = [i if 1960 < i < 2019 else np.nan for i in X2['construction_year']]
  indicator = MissingIndicator()
  trash_array = indicator.fit_transform(X2[numericals]) # Bool array
  trash_array = trash_array.astype('float64')     # Float64 array

    # Create a titles for the columns in num_trashmarker
  trashy_names = [numericals[i] + '_trash' for i in indicator.features_]

    # Create num_trashmarker
  trash_df = pd.DataFrame(trash_array, columns=trashy_names)

    # I add trash_df to X2
  X2 = pd.concat([X2,trash_df], sort=False, axis=1)
    
    
    # Fixing the numerical columns.
    # ---------------------------------------------------------------
    # Whenever possible, a good replacement value for a NAN is the 
    # mean or median value for the geographic region around it.

    # Replaces the NANs in a ward with the mean of the other rows in that 
    # same ward. If all the rows in a ward are NANs, though, they remain.
  for feature in numericals:
      replacements = X2.groupby('ward')[feature].transform('mean')
      X2[feature] = X2[feature].fillna(replacements)

    # Replaces the NANs in a region with the mean of the other rows in that 
    # same region (which are much larger than wards)
  for feature in numericals:
      replacements = X2.groupby('region')[feature].transform('mean')
      X2[feature] = X2[feature].fillna(replacements)
    
    # Replaces any remaining NANs with the median value for the whole dataset
  for feature in numericals:
      replacements = X2[feature].median() # Single number, not array
      X2[feature] = X2[feature].fillna(replacements)
    
  return X2

def cleanup3(X):
  X2 = X.copy()
    
    # Create list of categorical features
  categoricals = X2.select_dtypes(exclude='number').columns.tolist()

    # Make all strings lowercase, to collapse together some of the categories
  X2[categoricals] = X2[categoricals].applymap(lambda x: x.lower())

    # Replace common NAN values
  nan_list = ['not known','unknown','none','-','##','not kno','unknown installer']
  X2 = X2.replace(nan_list, np.nan)

    # Any feature values with fewer than 100 rows gets turned into a NAN
  for feature in X2[categoricals]:
        # Determine which feature values to keep
      to_keep = X2[feature].value_counts()[X2[feature].value_counts() > 100].index.tolist()
        # Turn those into NANs (using a copy, to prevent warnings)
      feature_copy = X2[feature].copy()
      feature_copy[~feature_copy.isin(to_keep)] = np.nan
      X2[feature] = feature_copy

    # Fix all NANs
  X2[categoricals] = X2[categoricals].fillna('other')
    
    
  return X2

from sklearn.preprocessing import RobustScaler
def cleanup4(X):
    X2 = X.copy()
    
    garbage = ['longitude','latitude','construction_year_trash',
              'latitude_trash','gps_height_trash',
               'extraction_type_group','extraction_type_class',
               'region_code','waterpoint_type_group','source_type',
              'payment_type','quality_group','quantity_group']
    
    X2 = X2.drop(columns=garbage)
    
    X2['age'] = X2['date_recorded'] - X2['construction_year']

    numericals = ['amount_tsh',
                    'date_recorded',
                    'gps_height',
                    'num_private',
                    'population',
                    'construction_year',
                    'age']

    scaler = RobustScaler()
    nums_scaled = scaler.fit_transform(X2[numericals])
    nums_scaled = pd.DataFrame(nums_scaled, columns=numericals)
    X2[numericals] = nums_scaled
    
    return X2

In [0]:
nan_values_list = ['Not Known', 'Unknown', 'None', 'Not known', 'not known',
                  '-', 'unknown', 'Unknown Installer', '##', 'none', '0']

df = df.replace(nan_values_list, np.nan)
testdf = testdf.replace(nan_values_list, np.nan)


In [0]:
red_list = ['Government', 'Central government', 'Central Govt', 'GOVER', 'Gover',
            'Gove', 'Centr', 'Central Government', 'Wizara ya maji', 'Water', 
            'GOVERNMENT', 'Tanzania Government', 'GOVERN', 'central government',
           'ISF/Government', 'CENTRAL GOVERNMENT', 'MINISTRY OF WATER', 
            'Ministry of water engineer', 'Cental Government', 'ADRA/Government']

list2 = ['World vision', 'World Vision', 'WORLD VISION', 'World Vission', 
        'World Division', 'Government /World Vision', 'World visiin']


df2['installer'] = df2['installer'].replace(red_list, 'government')
df2['installer'] = df2['installer'].replace(list2, 'World Vision')


In [0]:
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*secondary.*$)', 'school')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*school.*$)', 'school')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*shule.*$)', 'school')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*sekondari.*$)', 'school')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*msingi.*$)', 'school')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*primary.*$)', 'school')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*clinic.*$)', 'health')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*hospital.*$)', 'health')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*zahanati.*$)', 'health')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*health.*$)', 'health')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*ccm.*$)', 'official')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*office.*$)', 'official')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*kijiji.*$)', 'official')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*ofis.*$)', 'official')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*idara.*$)', 'offical')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*maziwa.*$)', 'farm')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*farm.*$)', 'farm')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*maji.*$)', 'pump')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*water.*$)', 'pump')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*pump house.*$)', 'pump')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*pump.*$)', 'pump')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*bombani.*$)', 'pump')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*center.*$)', 'center')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*madukani.*$)', 'center')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*sokoni.*$)', 'center')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*market.*$)', 'center')
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*kwa.*$)', 'name')
value_counts = df2['wpt_name'].value_counts()
to_remove = value_counts[value_counts <= 300].index
df2['wpt_name'].replace(to_remove, 'other', inplace=True)
df2['wpt_name'] = df2.wpt_name.str.replace(r'((?i)^.*none.*$)', 'other')